In [ ]:
import os
import rospy
import jupyros as jr
from jupyros import ros3d

from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Twist, Point

from move_base_msgs.msg import MoveBaseActionGoal
from actionlib_msgs.msg import GoalID

from IPython.display import clear_output
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Button, Layout, IntText, FloatText, ButtonStyle, GridBox, VBox, HBox, interact, widgets, interactive, fixed, interact_manual

wannago_x = -5.3
wannago_y = -5.3
# to initiate user desired values

reached_target = 0
unreached_target = 0

counter = 0
cnt = 0

position_ = Point()
# to compare goal and actual pos of robot

vel = Twist()
# to send to cmd_vel

velraw_ = Twist()
velraw_.linear.x = 0.0
velraw_.angular.z = 0.0
# to send to cmd_vel_raw

msg_cancel = GoalID()
# to cancel goal

# global publishers. pub3 is for manipulating user input.
pub  = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
pub3 = rospy.Publisher('/cmd_vel_raw', Twist, queue_size=1) #bunu ekledim
pub4 = rospy.Publisher('move_base/cancel', GoalID, queue_size=1) #bunu ekledim
pub5 = rospy.Publisher('move_base/goal', MoveBaseActionGoal, queue_size=1)

def service_func():
    # the main function. initiates communication with ros, and welcomes user and initiates button interface.
    rospy.init_node('jupyter_node')
    print('Welcome! Please choose a driving mode below.')
    button_interface0()
    
def button_interface0():
    # For welcome screen
    b1 = widgets.Button(description='Mod1: Auto-Drive', layout=Layout(width='50%', align="center", grid_area='b1'), style=ButtonStyle(button_color='lightblue'))
    b2 = widgets.Button(description='Mod2: User-Drive', layout=Layout(width='50%', grid_area='b2'), style=ButtonStyle(button_color='lightblue'))
    b3 = widgets.Button(description='Mod3: Assisted-Drive', layout=Layout(width='50%', grid_area='b3'), style=ButtonStyle(button_color='lightblue'))
    
    a = VBox(children=[b1, b2, b3])
    out = widgets.Output()
    
    display(a, out)
    
    b1.on_click(mod1)
    b2.on_click(mod2)
    b3.on_click(mod3)

def button_interface():
    # For second mod, user-drive. Buttons with arrows are initiated.
    bf = widgets.Button(description='▲', layout=Layout(width='90%', grid_area='bf'), style=ButtonStyle(button_color='orange'))
    bright = widgets.Button(description='▶', layout=Layout( width='90%', align="center", grid_area='bright'), style=ButtonStyle(button_color='orange'))
    bleft = widgets.Button(description='◀', layout=Layout(width='90%', grid_area='bleft'), style=ButtonStyle(button_color='orange'))
    b0 = widgets.Button(description='O', layout=Layout(width='90%', grid_area='b0'), style=ButtonStyle(button_color='lightblue'))
    bb = widgets.Button(description='▼', layout=Layout(width='90%', grid_area='bb'), style=ButtonStyle(button_color='orange'))
    
    a = GridBox(children=[bf, bright, bleft, b0, bf, bb], layout=Layout(width='50%', grid_template_rows='auto auto', grid_template_columns='10% 10% 10%', grid_template_areas='''" . bf . " "bleft b0 bright " " . bb . " '''))
    
    out2=widgets.Output()
    display(a,out2)
    
    # Button for Return to Main Menu option
    b1 = widgets.Button(description='Return to Main Menu', layout=Layout(width='50%', align="center", grid_area='b1'), style=ButtonStyle(button_color='lightblue'))
    
    a2 = VBox(children=[b1])
    out = widgets.Output()
    
    display(a2, out)
    
    b1.on_click(main_menu_clicked)
    bf.on_click(bf_on_button_clicked)
    bright.on_click(bright_on_button_clicked)
    bleft.on_click(bleft_on_button_clicked)
    b0.on_click(b0_on_button_clicked)
    bb.on_click(bb_on_button_clicked)

def button_interface2():
    # For third mod, assisted-drive. The same as initial one. Buttons are assigned to different functions to manipulate input.
    bf2 = widgets.Button(description='▲', layout=Layout(width='90%', grid_area='bf'), style=ButtonStyle(button_color='orange'))
    bright2 = widgets.Button(description='▶', layout=Layout( width='90%', align="center", grid_area='bright'), style=ButtonStyle(button_color='orange'))
    bleft2 = widgets.Button(description='◀', layout=Layout(width='90%', grid_area='bleft'), style=ButtonStyle(button_color='orange'))
    b02 = widgets.Button(description='O', layout=Layout(width='90%', grid_area='b0'), style=ButtonStyle(button_color='lightblue'))
    bb2 = widgets.Button(description='▼', layout=Layout(width='90%', grid_area='bb'), style=ButtonStyle(button_color='orange'))
    
    a2 = GridBox(children=[bf2, bright2, bleft2, b02, bf2, bb2], layout=Layout(width='50%', grid_template_rows='auto auto', grid_template_columns='10% 10% 10%', grid_template_areas='''" . bf . " "bleft b0 bright " " . bb . " '''))
    
    out22=widgets.Output()
    display(a2,out22)
    
    # Button for Return to Main Menu option
    b1 = widgets.Button(description='Return to Main Menu', layout=Layout(width='50%', align="center", grid_area='b1'), style=ButtonStyle(button_color='lightblue'))
    
    a5 = VBox(children=[b1])
    out = widgets.Output()
    
    display(a5, out)
    
    b1.on_click(main_menu_clicked)
    
    bf2.on_click(bf2_on_button_clicked)
    bright2.on_click(bright2_on_button_clicked)
    bleft2.on_click(bleft2_on_button_clicked)
    b02.on_click(b02_on_button_clicked)
    bb2.on_click(bb2_on_button_clicked)

def main_menu_clicked(b):
    global counter
    counter = 0
    clear_output()
    button_interface0()
    
def mod1(b):
    # mod1, auto-drive.
    clear_output()
    global x_coord, y_coord
    print('You chose Mod1: Auto-Drive!')
    
    x_coord = FloatText(value = -4.0, description = 'x coord to go:', step = '0.1')
    y_coord = FloatText(value =  4.0, description = 'y coord to go:', step = '0.1')
    
    display(x_coord, y_coord)
    
    b1 = widgets.Button(description='Lets go!')
    
    out1=widgets.Output()
    display(b1,out1)
    
    b1.on_click(b1_on_button_clicked)

def mod2(b):
    # mod2, user-drive. taking subscribe out of function caused problem. so kept it here.
    clear_output()
    print('You chose Mod2: User-Drive!')
    button_interface()
    rospy.Subscriber('/cmd_vel', Twist, clbck)
    
def mod3(b):
    # mod3, assisted-drive. subs are put here due to several errors.
    clear_output()
    print('You chose Mod3: Assisted-Drive!')
    rospy.Subscriber('/cmd_vel_raw', Twist, clbck3)
    rospy.Subscriber('/scan', LaserScan, clbck_laser)
    button_interface2()

def bf2_on_button_clicked(b):
    # mod3 button. output of this click is sent to /cmd_vel_raw. it will be modified later, if there is an obtacle in intended direction.
    global velraw_
    clear_output()
    print('You chose Mod3: Assisted-Drive!')
    button_interface2()
    velraw_.linear.x = velraw_.linear.x + 0.5
    velraw_.angular.z = 0.0
    print("Forward! Your linear velocity is")
    print(velraw_.linear.x)
    pub3.publish(velraw_)
    
def bright2_on_button_clicked(b):
    # mod3 button. output of this click is sent to /cmd_vel_raw. it will be modified later, if there is an obtacle in intended direction.
    global velraw_
    clear_output()
    print('You chose Mod3: Assisted-Drive!')
    button_interface2()
    velraw_.angular.z = velraw_.angular.z-0.5
    print("Right! Your side velocity is")
    print(velraw_.angular.z)
    pub3.publish(velraw_)
    
def bleft2_on_button_clicked(b):
    # mod3 button. output of this click is sent to /cmd_vel_raw. it will be modified later, if there is an obtacle in intended direction.
    global velraw_
    clear_output()
    print('You chose Mod3: Assisted-Drive!')
    button_interface2()
    velraw_.angular.z = velraw_.angular.z+0.5
    print("Left! Your side velocity is")
    print(velraw_.angular.z)
    pub3.publish(velraw_)
    
def b02_on_button_clicked(b):
    # mod3 button. output of this click is sent to /cmd_vel_raw. it will be modified later, if there is an obtacle in intended direction.
    global velraw_
    clear_output()
    print('You chose Mod3: Assisted-Drive!')
    button_interface2()
    print("Stay! Your velocity is 0")
    velraw_.linear.x = 0.0
    velraw_.angular.z = 0.0
    pub3.publish(velraw_)
    
def bb2_on_button_clicked(b):
    # mod3 button. output of this click is sent to /cmd_vel_raw. it will be modified later, if there is an obtacle in intended direction.
    global velraw_
    clear_output()
    print('You chose Mod3: Assisted-Drive!')
    button_interface2()
    velraw_.linear.x = velraw_.linear.x-0.5
    pub3.publish(velraw_)

def b1_on_button_clicked(b):
    # mod1 'Let Go' button, to send desired coordinates to MoveAction
    global wannago_x, wannago_y, counter
    wannago_x = x_coord.value
    wannago_y = y_coord.value
    
    move_msg = MoveBaseActionGoal()
    
    move_msg.goal.target_pose.pose.position.x = wannago_x
    move_msg.goal.target_pose.pose.position.y = wannago_y
    move_msg.goal.target_pose.header.frame_id = 'map'
    move_msg.goal.target_pose.pose.orientation.w = 1

    clear_output()
    print('You chose Mod1: Auto-Drive!')
    print("\nGoal is entered as",wannago_x,",", wannago_y, "\n\nOn the way!")
    
    bbb = widgets.Button(description='Cancel Goal',  layout=Layout(width='50%', align="center", grid_area='bbb'), style=ButtonStyle(button_color='lightblue'))
    a1 = VBox(children=[bbb])
    out1bb=widgets.Output()
    display(a1,out1bb)
    bbb.on_click(bbb_on_button_clicked)

    # Button for Return to Main Menu option
    b1 = widgets.Button(description='Return to Main Menu to Enter A New Goal', layout=Layout(width='50%', align="center", grid_area='b1'), style=ButtonStyle(button_color='lightblue'))

    a2 = VBox(children=[b1])
    out = widgets.Output()

    display(a2, out)

    b1.on_click(main_menu_clicked)
    pub5.publish(move_msg)

def bbb_on_button_clicked(b):
    # mod1 'Cancel Target' button, to cancel the goal
    global unreached_target
    unreached_target = unreached_target + 1
    pub4.publish(msg_cancel)
    clear_output()
    print("Number of reached target is",reached_target,", unreached target is",unreached_target,"so far.")
    service_func()    
    
def bf_on_button_clicked(b):
    # mod2 button, forward
    global vel
    clear_output()
    print('You chose Mod2: User-Drive!')
    button_interface()
    vel.linear.x = vel.linear.x + 0.5
    vel.angular.z = 0.0
    print("Forward! Your linear velocity is")
    print(vel.linear.x)
    pub.publish(vel)
    
def bright_on_button_clicked(b):
    # mod2 button, right
    global vel
    clear_output()
    print('You chose Mod2: User-Drive!')
    button_interface()
    vel.angular.z = vel.angular.z-0.5
    print("Right! Your side velocity is")
    print(vel.angular.z)
    pub.publish(vel)
    
def bleft_on_button_clicked(b):
    # mod2 button, left
    global vel
    clear_output()
    print('You chose Mod2: User-Drive!')
    button_interface()
    vel.angular.z = vel.angular.z+0.5
    print("Left! Your side velocity is")
    print(vel.angular.z)
    pub.publish(vel)
    
def b0_on_button_clicked(b):
    # mod2 button, stay
    global vel
    clear_output()
    print('You chose Mod2: User-Drive!')
    button_interface()
    print("Stay! Your velocity is 0")
    vel.linear.x = 0.0
    vel.angular.z = 0.0
    pub.publish(vel)
    
def bb_on_button_clicked(b):
    # mod2 button, backwards
    global vel
    clear_output()
    print('You chose Mod2: User-Drive!')
    button_interface()
    vel.linear.x = vel.linear.x - 0.5
    print(velraw_.linear.x)
    pub.publish(vel)
    
def clbck(msg):
    # cmd_vel callback for mod2
    global vel
    vel.linear.x  = msg.linear.x
    vel.angular.z = msg.angular.z
    
def clbck2(msg):
    # mod1: auto-drive callback, get x and y pos for goal reach determination
    global position_, counter, reached_target, unreached_target
    global cnt
    
    position_ = msg.pose.pose.position
    threshold = 0.4

    if abs(wannago_x-position_.x) <= threshold and abs(wannago_y-position_.y) <= threshold and counter == 0:
        print("Number of reached target is",reached_target,", unreached target is",unreached_target,"so far.")
        pub4.publish(msg_cancel)
        reached_target = reached_target + 1
        counter = counter + 1

rospy.Subscriber('/odom', Odometry, clbck2)

def clbck3(msg):
    # Callback for reading from raw user velocity desires
    global velraw_
    velraw_ = msg

def clbck_laser(msg):
    # callback for laser reading, to determine if there is an obstacle in user's desired direction.
    regions = {
        'right':  min(min(msg.ranges[0:143]), 10),
        'fright': min(min(msg.ranges[144:287]), 10),
        'front':  min(min(msg.ranges[288:431]), 10),
        'fleft':  min(min(msg.ranges[432:575]), 10),
        'left':   min(min(msg.ranges[576:719]), 10),
    }
    take_action(regions)
    
def take_action(regions):
    """
    The function decides to whether keep the user velocity to the /cmd_vel or not. 5 region of the /scan readings are checked and if user wants to drive in the direction of a close obstacle, the velocity in that direction is set to zero. It is then sent to /cmd_vel.
    
    Args:
    	regions(dictionary): The parsed /scan readings to 5 regions. Every key has minimum of each region's closest obstacle distance as value.
    """
    msg = Twist()
    linear_x = velraw_.linear.x
    angular_z = velraw_.angular.z

    state_description = ''

    if regions['front'] < 1 and regions['fleft'] > 1 and regions['fright'] > 1:
        state_description = 'case 2 - front'
        if linear_x > 0:
            linear_x = 0
    elif regions['front'] > 1 and regions['fleft'] > 1 and regions['fright'] < 1:
        state_description = 'case 3 - fright'
        if angular_z < 0:
            angular_z = 0
    elif regions['front'] > 1 and regions['fleft'] < 1 and regions['fright'] > 1:
        state_description = 'case 4 - fleft'
        if angular_z > 0:
            angular_z = 0
    elif regions['front'] < 1 and regions['fleft'] > 1 and regions['fright'] < 1:
        state_description = 'case 5 - front and fright'
        if linear_x > 0 or angular_z < 0:
            linear_x = 0
            angular_z = 0
    elif regions['front'] < 1 and regions['fleft'] < 1 and regions['fright'] > 1:
        state_description = 'case 6 - front and fleft'
        if linear_x > 0 or angular_z > 0:
            linear_x = 0
            angular_z = 0
    elif regions['front'] < 1 and regions['fleft'] < 1 and regions['fright'] < 1:
        state_description = 'case 7 - front and fleft and fright'
        if linear_x > 0:
            linear_x = 0
    elif regions['front'] > 1 and regions['fleft'] < 1 and regions['fright'] < 1:
        state_description = 'case 8 - fleft and fright'
        if angular_z != 0:
            angular_z = 0
    else:
        state_description = 'unknown case'

    msg.linear.x = linear_x
    msg.angular.z = angular_z
    pub.publish(msg)
    
if __name__ == '__main__':
    service_func()

In [ ]:
v = ros3d.Viewer()
rc = ros3d.ROSConnection(url="ws://localhost:9090")
tf_client = ros3d.TFClient(ros=rc, fixed_frame='map')
laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client)
map_view = ros3d.OccupancyGrid(topic="/map", ros=rc, tf_client=tf_client)
path = ros3d.Path(topic="/move_base/NavfnROS/plan", ros=rc, 
tf_client=tf_client)
urdf = ros3d.URDFModel(ros=rc, tf_client=tf_client, 
path=os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000'))
g = ros3d.GridModel()
v.objects = [g, laser_view, map_view, path, urdf]
v